# 載入必要套件

In [ ]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
import re
import numpy as np
import torch
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from lpips import LPIPS

# 定義loss計算方法

## PSNR

In [ ]:
def calculate_psnr(img1, img2):
    """Calculate PSNR between two images."""
    return psnr(img1, img2, data_range=1.0)

## SSIM

In [ ]:
def calculate_ssim(img1, img2):
    """Calculate SSIM between two images."""
    return ssim(img1, img2, data_range=1.0, multichannel=True,channel_axis=-1)

## LPIPS

In [ ]:
# 初始化 LPIPS 度量器
lpips_fn = LPIPS(net='vgg').cuda()

def calculate_lpips(img1, img2):
    """Calculate LPIPS between two images."""
    img1 = img1.astype(np.float32)
    img2 = img2.astype(np.float32)

    img1_tensor = torch.tensor(img1).permute(2, 0, 1).unsqueeze(0).cuda()
    img2_tensor = torch.tensor(img2).permute(2, 0, 1).unsqueeze(0).cuda()
    return lpips_fn(img1_tensor, img2_tensor).item()

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 76.8MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/vgg.pth


/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), st

## 一對圖片的處理流程函數

In [ ]:
def process_and_calculate_losses(image_path1, image_path2):
    """Process two image paths and calculate PSNR, SSIM, and LPIPS."""
    def preprocess_image(image_path):
        # Load image and convert to NumPy array normalized to [0, 1]
        img = Image.open(image_path).convert("RGB")
        img_np = np.array(img) / 255.0

        # 顯示圖像大小
        #print(img_np.shape)
        return img_np

    # Preprocess both images
    img1 = preprocess_image(image_path1)
    img2 = preprocess_image(image_path2)

    # Calculate losses
    psnr_value = calculate_psnr(img1, img2)
    ssim_value = calculate_ssim(img1, img2)
    lpips_value = calculate_lpips(img1, img2)
    #lpips_value = 0

    return psnr_value, ssim_value, lpips_value


In [ ]:
#test
image_path1 = '/content/drive/MyDrive/中興_深度學習/HW4/2-MADF/celeba/gt_results/celeba_test/000011.png'
image_path2 = '/content/drive/MyDrive/中興_深度學習/HW4/2-MADF/celeba/results/celeba_test/000011.png'
psnr_val, ssim_val, lpips_val = process_and_calculate_losses(image_path1, image_path2)
print(f"PSNR: {psnr_val}, SSIM: {ssim_val}, LPIPS: {lpips_val}")

PSNR: 10.10176609242681, SSIM: 0.5461050157569517, LPIPS: 0.5423887372016907


# 2-MADF

## 批處理函數

In [ ]:
def batch_process_and_calculate_losses(folder1, folder2):
    """Batch process two folders and calculate average PSNR, SSIM, and LPIPS."""
    psnr_values = []
    ssim_values = []
    lpips_values = []

    # Get all filenames in folder1 and folder2
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))

    # Ensure the filenames match
    common_files = set(files1) & set(files2)
    if not common_files:
        raise ValueError("No matching files found between the two folders.")

    for filename in common_files:
        image_path1 = os.path.join(folder1, filename)
        image_path2 = os.path.join(folder2, filename)

        psnr_value, ssim_value, lpips_value = process_and_calculate_losses(image_path1, image_path2)
        psnr_values.append(psnr_value)
        ssim_values.append(ssim_value)
        lpips_values.append(lpips_value)

    # Calculate average losses
    psnr_avg = np.mean(psnr_values)
    ssim_avg = np.mean(ssim_values)
    lpips_avg = np.mean(lpips_values)

    return psnr_avg, ssim_avg, lpips_avg

In [ ]:
#多線程

import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor


def process_file_pair(filename, folder1, folder2):
    """Process a single file pair and return the losses."""
    image_path1 = os.path.join(folder1, filename)
    image_path2 = os.path.join(folder2, filename)
    return process_and_calculate_losses(image_path1, image_path2)

def batch_process_and_calculate_losses_nthreads(folder1, folder2, num_threads=4):
    """Batch process two folders and calculate average PSNR, SSIM, and LPIPS using multithreading."""
    psnr_values = []
    ssim_values = []
    lpips_values = []

    # Get all filenames in folder1 and folder2
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))

    # Ensure the filenames match
    common_files = sorted(set(files1) & set(files2))
    if not common_files:
        raise ValueError("No matching files found between the two folders.")

    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Map each common file to the process_file_pair function
        results = executor.map(
            process_file_pair, common_files,
            [folder1] * len(common_files),
            [folder2] * len(common_files)
        )

        # Collect the results
        for psnr_value, ssim_value, lpips_value in results:
            psnr_values.append(psnr_value)
            ssim_values.append(ssim_value)
            lpips_values.append(lpips_value)

    # Calculate average losses
    psnr_avg = np.mean(psnr_values)
    ssim_avg = np.mean(ssim_values)
    lpips_avg = np.mean(lpips_values)

    return psnr_avg, ssim_avg, lpips_avg

# Example usage:
# folder1 = "path_to_folder1"
# folder2 = "path_to_folder2"
# psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses(folder1, folder2, num_threads=8)
# print(f"PSNR: {psnr_avg:.2f}, SSIM: {ssim_avg:.4f}, LPIPS: {lpips_avg:.4f}")


## 計算loss

In [ ]:
#定義路徑相關變數
set1 = '/content/drive/MyDrive/中興_深度學習/HW4/2-MADF/place2'
set2 = '/content/drive/MyDrive/中興_深度學習/HW4/2-MADF/celeba'
path_ans = 'gt_results'
path_generate = 'results'
test_datasets = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]



In [ ]:
#place2
print(f"Processing train set: place2")
for dataset in test_datasets:

  folder1 = os.path.join(set1, path_ans, dataset)
  folder2 = os.path.join(set1, path_generate, dataset)
  print(f"Processing dataset: {dataset}")

  psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses(folder1, folder2)
  print(f"  Average PSNR: {psnr_avg:.4f}")
  print(f"  Average SSIM: {ssim_avg:.4f}")
  print(f"  Average LPIPS: {lpips_avg:.4f}")

Processing train set: place2
Processing dataset: FFHQ_test
  Average PSNR: 16.2676
  Average SSIM: 0.6152
  Average LPIPS: 0.4608
Processing dataset: Shunghaitech_test
  Average PSNR: 17.2293
  Average SSIM: 0.6463
  Average LPIPS: 0.4266
Processing dataset: celeba_test
  Average PSNR: 15.5949
  Average SSIM: 0.6225
  Average LPIPS: 0.4362
Processing dataset: paris_test
  Average PSNR: 18.2173
  Average SSIM: 0.5906
  Average LPIPS: 0.4661
Processing dataset: place2_chruch_indoor_test
  Average PSNR: 17.0585
  Average SSIM: 0.5482
  Average LPIPS: 0.4663
Processing dataset: place2_chruch_ourdoor_test
  Average PSNR: 16.2179
  Average SSIM: 0.5893
  Average LPIPS: 0.4594


In [ ]:
#celeba
print(f"Processing train set: celeba")
for dataset in test_datasets:

  folder1 = os.path.join(set2, path_ans, dataset)
  folder2 = os.path.join(set2, path_generate, dataset)
  print(f"Processing dataset: {dataset}")

  psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(folder1, folder2)
  print(f"  Average PSNR: {psnr_avg:.4f}")
  print(f"  Average SSIM: {ssim_avg:.4f}")
  print(f"  Average LPIPS: {lpips_avg:.4f}")

Processing train set: celeba
Processing dataset: FFHQ_test
  Average PSNR: 16.1671
  Average SSIM: 0.6163
  Average LPIPS: 0.4540
Processing dataset: Shunghaitech_test
  Average PSNR: 16.6142
  Average SSIM: 0.6353
  Average LPIPS: 0.4474
Processing dataset: celeba_test
  Average PSNR: 15.4477
  Average SSIM: 0.6186
  Average LPIPS: 0.4387
Processing dataset: paris_test
  Average PSNR: 17.3382
  Average SSIM: 0.5747
  Average LPIPS: 0.4930
Processing dataset: place2_chruch_indoor_test
  Average PSNR: 16.6357
  Average SSIM: 0.5402
  Average LPIPS: 0.4829
Processing dataset: place2_chruch_ourdoor_test
  Average PSNR: 15.4119
  Average SSIM: 0.5758
  Average LPIPS: 0.4816


# 4-AOT-GAN

## 批處理函數

In [ ]:
# 多線程
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor


def process_file_pair(answer_file, answer_folder, generated_folder):
    """Process a single file pair and return the losses."""
    name_without_ext = os.path.splitext(answer_file)[0]  # Remove file extension
    generated_file = f"{name_without_ext}_pred.png"
    answer_path = os.path.join(answer_folder, answer_file)
    generated_path = os.path.join(generated_folder, generated_file)

    if not os.path.exists(generated_path):
        raise FileNotFoundError(f"Generated file not found: {generated_path}")

    # Replace this function with the actual processing logic
    return process_and_calculate_losses(answer_path, generated_path)


def batch_process_and_calculate_losses_nthreads(answer_folder, generated_folder, num_threads=4):
    """Batch process two folders and calculate average PSNR, SSIM, and LPIPS using multithreading."""
    psnr_values = []
    ssim_values = []
    lpips_values = []

    # Get all filenames in the answer folder
    answer_files = sorted(
        [f for f in os.listdir(answer_folder) if f.endswith((".jpg", ".png"))]
    )

    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Map each answer file to the process_file_pair function
        results = executor.map(
            process_file_pair, answer_files,
            [answer_folder] * len(answer_files),
            [generated_folder] * len(answer_files)
        )

        # Collect the results
        for psnr_value, ssim_value, lpips_value in results:
            psnr_values.append(psnr_value)
            ssim_values.append(ssim_value)
            lpips_values.append(lpips_value)

    # Calculate average losses
    psnr_avg = np.mean(psnr_values)
    ssim_avg = np.mean(ssim_values)
    lpips_avg = np.mean(lpips_values)

    return psnr_avg, ssim_avg, lpips_avg


# Example usage:
# answer_folder = "path_to_answer_folder"
# generated_folder = "path_to_generated_folder"
# psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(answer_folder, generated_folder, num_threads=8)
# print(f"PSNR: {psnr_avg:.2f}, SSIM: {ssim_avg:.4f}, LPIPS: {lpips_avg:.4f}")


## 計算loss

In [ ]:
#定義路徑相關變數
set1 = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/place2/output'
set2 = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba/output'
path_ans = '/content/drive/MyDrive/中興_深度學習/HW4/test'

test_datasets = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]



In [ ]:
#place2
print(f"Processing train set: place2")
for dataset in test_datasets:

  folder1 = os.path.join(path_ans, dataset)
  folder2 = os.path.join(set1, dataset)
  print(f"Processing dataset: {dataset}")

  psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(folder1, folder2)
  print(f"  Average PSNR: {psnr_avg:.4f}")
  print(f"  Average SSIM: {ssim_avg:.4f}")
  print(f"  Average LPIPS: {lpips_avg:.4f}")

Processing train set: place2
Processing dataset: FFHQ_test
  Average PSNR: 8.0582
  Average SSIM: 0.1866
  Average LPIPS: 0.6846
Processing dataset: Shunghaitech_test
  Average PSNR: 7.6069
  Average SSIM: 0.1865
  Average LPIPS: 0.6703
Processing dataset: celeba_test
  Average PSNR: 7.9051
  Average SSIM: 0.2024
  Average LPIPS: 0.6577
Processing dataset: paris_test
  Average PSNR: 9.0108
  Average SSIM: 0.1558
  Average LPIPS: 0.6804
Processing dataset: place2_chruch_indoor_test
  Average PSNR: 8.9498
  Average SSIM: 0.1530
  Average LPIPS: 0.6693
Processing dataset: place2_chruch_ourdoor_test
  Average PSNR: 7.9198
  Average SSIM: 0.1720
  Average LPIPS: 0.6707


In [ ]:
#celeba
print(f"Processing train set: celeba")
for dataset in test_datasets:

  folder1 = os.path.join(path_ans, dataset)
  folder2 = os.path.join(set2, dataset)
  print(f"Processing dataset: {dataset}")

  psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(folder1, folder2)
  print(f"  Average PSNR: {psnr_avg:.4f}")
  print(f"  Average SSIM: {ssim_avg:.4f}")
  print(f"  Average LPIPS: {lpips_avg:.4f}")

Processing train set: celeba
Processing dataset: FFHQ_test
  Average PSNR: 9.4935
  Average SSIM: 0.2123
  Average LPIPS: 0.6838
Processing dataset: Shunghaitech_test
  Average PSNR: 9.2613
  Average SSIM: 0.2224
  Average LPIPS: 0.6544
Processing dataset: celeba_test
  Average PSNR: 9.1209
  Average SSIM: 0.2230
  Average LPIPS: 0.6557
Processing dataset: paris_test
  Average PSNR: 10.0699
  Average SSIM: 0.1889
  Average LPIPS: 0.6677
Processing dataset: place2_chruch_indoor_test
  Average PSNR: 9.7375
  Average SSIM: 0.1717
  Average LPIPS: 0.6549
Processing dataset: place2_chruch_ourdoor_test


FileNotFoundError: Generated file not found: /content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba/output/place2_chruch_ourdoor_test/00000291_pred.png

In [ ]:
#定義路徑相關變數
set1 = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/place2/output'
set2 = '/content/drive/MyDrive/中興_深度學習/HW4/4-AOT-GAN/celeba/output'
path_ans = '/content/drive/MyDrive/中興_深度學習/HW4/test'

test_datasets = ["place2_chruch_ourdoor_test"]

#celeba redo of place2_chruch_ourdoor_test
print(f"Processing train set: celeba")
for dataset in test_datasets:

  folder1 = os.path.join(path_ans, dataset)
  folder2 = os.path.join(set2, dataset)
  print(f"Processing dataset: {dataset}")

  psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(folder1, folder2)
  print(f"  Average PSNR: {psnr_avg:.4f}")
  print(f"  Average SSIM: {ssim_avg:.4f}")
  print(f"  Average LPIPS: {lpips_avg:.4f}")

Processing train set: celeba
Processing dataset: place2_chruch_ourdoor_test
  Average PSNR: 9.1287
  Average SSIM: 0.1996
  Average LPIPS: 0.6568


# 5-TFill

## 批處理函數

In [ ]:
# 多線程
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor


def process_file_pair(answer_file, answer_folder, generated_folder):
    """Process a single file pair and return the losses."""
    name_without_ext = os.path.splitext(answer_file)[0]  # Remove file extension
    generated_file = f"{name_without_ext}_0.png"
    answer_path = os.path.join(answer_folder, answer_file)
    generated_path = os.path.join(generated_folder, generated_file)

    if not os.path.exists(generated_path):
        raise FileNotFoundError(f"Generated file not found: {generated_path}")

    # Replace this function with the actual processing logic
    return process_and_calculate_losses(answer_path, generated_path)


def batch_process_and_calculate_losses_nthreads(answer_folder, generated_folder, num_threads=4):
    """Batch process two folders and calculate average PSNR, SSIM, and LPIPS using multithreading."""
    psnr_values = []
    ssim_values = []
    lpips_values = []

    # Get all filenames in the answer folder
    answer_files = sorted(
        [f for f in os.listdir(answer_folder) if f.endswith((".jpg", ".png"))]
    )

    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Map each answer file to the process_file_pair function
        results = executor.map(
            process_file_pair, answer_files,
            [answer_folder] * len(answer_files),
            [generated_folder] * len(answer_files)
        )

        # Collect the results
        for psnr_value, ssim_value, lpips_value in results:
            psnr_values.append(psnr_value)
            ssim_values.append(ssim_value)
            lpips_values.append(lpips_value)

    # Calculate average losses
    psnr_avg = np.mean(psnr_values)
    ssim_avg = np.mean(ssim_values)
    lpips_avg = np.mean(lpips_values)

    return psnr_avg, ssim_avg, lpips_avg


# Example usage:
# answer_folder = "path_to_answer_folder"
# generated_folder = "path_to_generated_folder"
# psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(answer_folder, generated_folder, num_threads=8)
# print(f"PSNR: {psnr_avg:.2f}, SSIM: {ssim_avg:.4f}, LPIPS: {lpips_avg:.4f}")


## 計算loss

In [ ]:
#定義路徑相關變數
set1 = '/content/drive/MyDrive/中興_深度學習/HW4/5-TFill/place2/result'
set2 = '/content/drive/MyDrive/中興_深度學習/HW4/5-TFill/celeba/result'
test_datasets = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]

path_ans = '/content/drive/MyDrive/中興_深度學習/HW4/test'

In [ ]:
#place2
print(f"Processing train set: place2")
for dataset in test_datasets:

  folder1 = os.path.join(path_ans, dataset)
  folder2 = os.path.join(set1, dataset)

  print(f"Processing dataset: {dataset}")

  psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(folder1, folder2)
  print(f"  Average PSNR: {psnr_avg:.4f}")
  print(f"  Average SSIM: {ssim_avg:.4f}")
  print(f"  Average LPIPS: {lpips_avg:.4f}")

Processing train set: place2
Processing dataset: FFHQ_test
  Average PSNR: 12.8075
  Average SSIM: 0.6288
  Average LPIPS: 0.4387
Processing dataset: Shunghaitech_test
  Average PSNR: 13.9891
  Average SSIM: 0.6521
  Average LPIPS: 0.3821
Processing dataset: celeba_test
  Average PSNR: 13.1856
  Average SSIM: 0.6376
  Average LPIPS: 0.3836
Processing dataset: paris_test
  Average PSNR: 15.1313
  Average SSIM: 0.6543
  Average LPIPS: 0.3958
Processing dataset: place2_chruch_indoor_test
  Average PSNR: 12.0854
  Average SSIM: 0.6332
  Average LPIPS: 0.3779
Processing dataset: place2_chruch_ourdoor_test
  Average PSNR: 12.3795
  Average SSIM: 0.6409
  Average LPIPS: 0.3806


In [ ]:
#celeba
print(f"Processing train set: celeba")
for dataset in test_datasets:

  folder1 = os.path.join(path_ans, dataset)
  folder2 = os.path.join(set2, dataset)

  print(f"Processing dataset: {dataset}")

  psnr_avg, ssim_avg, lpips_avg = batch_process_and_calculate_losses_nthreads(folder1, folder2)
  print(f"  Average PSNR: {psnr_avg:.4f}")
  print(f"  Average SSIM: {ssim_avg:.4f}")
  print(f"  Average LPIPS: {lpips_avg:.4f}")

Processing train set: celeba
Processing dataset: FFHQ_test
  Average PSNR: 13.1677
  Average SSIM: 0.6361
  Average LPIPS: 0.4141
Processing dataset: Shunghaitech_test
  Average PSNR: 15.0014
  Average SSIM: 0.6642
  Average LPIPS: 0.3515
Processing dataset: celeba_test
  Average PSNR: 13.9048
  Average SSIM: 0.6477
  Average LPIPS: 0.3686
Processing dataset: paris_test
  Average PSNR: 14.4061
  Average SSIM: 0.6525
  Average LPIPS: 0.3934
Processing dataset: place2_chruch_indoor_test
  Average PSNR: 13.5730
  Average SSIM: 0.6428
  Average LPIPS: 0.3666
Processing dataset: place2_chruch_ourdoor_test
  Average PSNR: 13.5541
  Average SSIM: 0.6544
  Average LPIPS: 0.4264
